0. Paste token in form below.
1. Install. 
2. If you see an error concerning tokenizers or sentencepiece not being installed properly, simply press **runtime** and **restart and run all** and it will work.



In [ ]:
token = "" # @param {"type": "string"}

In [ ]:

# !pip install -q relevanceai==2.5.4
# !pip install -q langdetect
# !pip install -q sentencepiece
# !pip install -q transformers

from relevanceai.utils import decode_workflow_token
from langdetect import detect_langs
import json
import requests
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

config = decode_workflow_token(token)
print(config)
fields = config['fields']
dataset_id = config['dataset_id']

from relevanceai import Client
try:
  import os
  os.environ['WORKFLOW_ID'] = job_id
except Exception as e:
  print(e)
client = Client(config['authorizationToken'])

lang_dict = requests.get(
    "https://gist.githubusercontent.com/boba-and-beer/b142b44e7c5120c5ce839e4b7f8f0247/raw/21b73269fecb67e871843086fe8ebe78d8b4c7a0/lang_isocode_mapping.json"
).json()

lang_to_normal = requests.get(
    "https://gist.githubusercontent.com/boba-and-beer/65bf4a0810d6492050577f1d2373d394/raw/d3817a37401b0776b8ed4bda1bbcf89e8375c747/lang_iso_2_mapping.json"
).json()

model_id = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_id)
tokenizer = MBart50TokenizerFast.from_pretrained(model_id)

fields = config['fields']

def translate_to_english(text):
  language = detect_langs(text)
  if language[0].lang == "en":
    return None, None
  supported_languages = []
  for lang in language:
    try:
      tokenizer.src_lang = lang_dict[lang.lang]
      if lang.lang in lang_to_normal:
        supported_languages.append(lang_to_normal[lang.lang])
      else:
        supported_languages.append(lang.lang)
      
      encoded_hi = tokenizer(text, return_tensors="pt")
      generated_tokens = model.generate(
          **encoded_hi,
          forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
      )
      translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      return supported_languages, translation[0]
      break
    except Exception as e:
      pass
      # print(e)
  
  return None, None

def translate_document(field, document):
  sup_lang, translation = translate_to_english(
      client.get_field(field, document)
  )
  client.set_field(
      "_translation_." + field, document, {"detectedLanguage": sup_lang, "translation": translation}
  )
  return document

def bulk_translate_documents(documents):
  for field in fields:
    [translate_document(field, d) for d in documents]
  return documents

ds = client.Dataset(config['dataset_id'])
ds.bulk_apply(
    bulk_translate_documents,
    select_fields=fields,
    filters=ds[fields[0]].exists()
)